# Pipeline de Transcription et Sous-titrage

**Module :** 04-Audio-Applications  
**Niveau :** Applications  
**Technologies :** Whisper (local + API), pyannote (diarisation), pysrt, GPT (LLM)  
**VRAM estimee :** ~12 GB  
**Duree estimee :** 55 minutes  

## Objectifs d'Apprentissage

- [ ] Transcrire un fichier audio avec horodatage (timestamps)
- [ ] Mettre en place un pipeline de transcription batch (plusieurs fichiers)
- [ ] Comprendre la diarisation de locuteurs (identification des intervenants)
- [ ] Generer des sous-titres aux formats SRT et VTT
- [ ] Resumer automatiquement des reunions (transcription -> LLM -> resume)
- [ ] Evaluer la qualite de transcription et appliquer du post-traitement

## Prerequis

- Notebooks Foundation (01-2 Whisper STT, 01-4 Whisper Local) completes
- Cle API OpenAI configuree (`OPENAI_API_KEY` dans `.env`)
- faster-whisper installe (transcription locale)
- Comprehension du STT et des formats de sous-titres

**Navigation :** [Index](../README.md) | [<< Precedent](04-1-Educational-Audio-Content.ipynb) | [Suivant >>](04-3-Music-Composition-Workflow.ipynb)

In [ ]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres transcription
whisper_model = "large-v3-turbo"   # Modele Whisper local
whisper_device = "cuda"            # "cuda" ou "cpu"
whisper_api_model = "whisper-1"    # Modele API OpenAI
default_language = "fr"            # Langue par defaut
llm_model = "gpt-4o-mini"         # Modele pour le resume

# Configuration pipeline
use_local_whisper = False          # Desactive Whisper local pour validation (problèmes CUDA/CUBLAS)
generate_audio = True              # Generer des fichiers audio de test
save_output_files = True           # Sauvegarder les transcriptions

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import time
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from io import BytesIO
import logging

from IPython.display import Audio, display, HTML

# Resolution GENAI_ROOT
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import (
            transcribe_openai, transcribe_local,
            synthesize_openai, get_audio_info
        )
        print("Helpers audio importes")
    except ImportError as e:
        print(f"Helpers audio non disponibles - mode autonome : {e}")

# Repertoires
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'transcription'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('transcription_pipeline')

print(f"Pipeline de Transcription et Sous-titrage")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Whisper : {'local (' + whisper_model + ')' if use_local_whisper else 'API (' + whisper_api_model + ')'}")
print(f"Sortie : {OUTPUT_DIR}")

In [ ]:
# Chargement de la configuration et validation
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve dans l'arborescence")

openai_key = os.getenv('OPENAI_API_KEY')

if not openai_key:
    if notebook_mode == "batch" and not generate_audio:
        openai_key = "dummy_key_for_validation"
        print("Mode batch sans generation : cle API ignoree")
    else:
        raise ValueError("OPENAI_API_KEY manquante dans .env")

from openai import OpenAI
client = OpenAI(api_key=openai_key)

# Validation Whisper local avec detection automatique du device et compute_type
whisper_local_available = False
actual_whisper_device = "cpu"  # Par defaut CPU pour eviter les erreurs CUDA
actual_compute_type = "int8"   # CPU utilise int8 ou float32

if use_local_whisper:
    try:
        from faster_whisper import WhisperModel
        
        # Tester si CUDA est disponible
        if whisper_device == "cuda":
            try:
                test_model = WhisperModel("tiny", device="cuda", compute_type="float16")
                del test_model
                actual_whisper_device = "cuda"
                actual_compute_type = "float16"
                print("CUDA disponible - faster-whisper utilisera GPU")
            except Exception as cuda_err:
                if "cublas" in str(cuda_err).lower() or "cuda" in str(cuda_err).lower():
                    print(f"Erreur CUDA detectee : basculement vers CPU")
                    actual_whisper_device = "cpu"
                    actual_compute_type = "int8"
                else:
                    raise
        else:
            actual_whisper_device = whisper_device
            actual_compute_type = "float16" if actual_whisper_device == "cuda" else "int8"
        
        whisper_local_available = True
        print(f"faster-whisper disponible (modele : {whisper_model}, device : {actual_whisper_device}, compute : {actual_compute_type})")
    except ImportError:
        print("faster-whisper non installe - basculement vers l'API")
        use_local_whisper = False
    except Exception as e:
        print(f"Erreur initialise Whisper : {e}")
        print(f"Basculement vers l'API OpenAI")
        use_local_whisper = False

# Generer des fichiers audio de test via TTS
test_audio_files = {}
if generate_audio and openai_key != "dummy_key_for_validation":
    print("\nGeneration de fichiers audio de test...")
    test_texts = {
        "presentation": (
            "Bonjour a tous. Bienvenue dans cette presentation sur l'intelligence artificielle. "
            "Aujourd'hui nous allons aborder trois sujets principaux. "
            "Premierement, les fondamentaux du machine learning. "
            "Deuxiemement, les reseaux de neurones. "
            "Et troisiemement, les applications pratiques."
        ),
        "reunion": (
            "Merci d'etre presents pour cette reunion d'equipe. "
            "Le premier point a l'ordre du jour concerne l'avancement du projet. "
            "Nous avons termine la phase de conception et nous passons maintenant au developpement. "
            "Le prochain jalon est prevu pour la fin du mois."
        )
    }

    for name, text in test_texts.items():
        filepath = OUTPUT_DIR / f"test_{name}.mp3"
        response = client.audio.speech.create(
            model="tts-1", voice="nova", input=text, response_format="mp3"
        )
        with open(filepath, 'wb') as f:
            f.write(response.content)
        test_audio_files[name] = filepath
        print(f"  {name} : {filepath.name} ({len(response.content)/1024:.1f} KB)")

print(f"\nConfiguration prete")

## Section 1 : Transcription avec horodatage

Une transcription de qualite necessite plus que le texte brut : les timestamps permettent de situer chaque segment dans le temps.

| Methode | Latence | Qualite | Cout | Timestamps |
|---------|---------|---------|------|------------|
| Whisper API | ~10s/min | Bonne | $0.006/min | Segment-level |
| Whisper local (large-v3-turbo) | ~3s/min (GPU) | Excellente | Gratuit | Word-level |
| Whisper local (tiny) | ~0.5s/min (GPU) | Correcte | Gratuit | Segment-level |

Whisper local offre des timestamps au niveau du mot, essentiels pour les sous-titres precis.

In [ ]:
# Transcription avec horodatage
print("TRANSCRIPTION AVEC HORODATAGE")
print("=" * 50)

transcription_results = {}

if generate_audio and test_audio_files:
    test_file = test_audio_files.get("presentation")

    if test_file and test_file.exists():
        print(f"Fichier : {test_file.name}")

        if use_local_whisper and whisper_local_available:
            # Transcription locale avec timestamps
            print(f"\nMethode : Whisper local ({whisper_model})")
            start_time = time.time()

            try:
                model = WhisperModel(whisper_model, device=actual_whisper_device, compute_type=actual_compute_type)
                segments, info = model.transcribe(
                    str(test_file),
                    language=default_language,
                    word_timestamps=True
                )
                segments_list = list(segments)
                transcription_time = time.time() - start_time

                print(f"Langue detectee : {info.language} (prob: {info.language_probability:.2f})")
                print(f"Duree audio : {info.duration:.1f}s")
                print(f"Temps de transcription : {transcription_time:.1f}s")
                print(f"Ratio temps reel : {info.duration/transcription_time:.1f}x")

                print(f"\nSegments avec timestamps :")
                full_text = ""
                for seg in segments_list:
                    print(f"  [{seg.start:6.1f}s - {seg.end:6.1f}s] {seg.text.strip()}")
                    full_text += seg.text.strip() + " "

                transcription_results["local"] = {
                    "text": full_text.strip(),
                    "segments": segments_list,
                    "time": transcription_time,
                    "duration": info.duration
                }
            except Exception as e:
                print(f"Erreur transcription locale : {e}")
                print(f"Basculement vers l'API OpenAI")
                use_local_whisper = False

        if not use_local_whisper or not whisper_local_available or "local" not in transcription_results:
            # Transcription via API
            print(f"\nMethode : API OpenAI ({whisper_api_model})")
            start_time = time.time()

            with open(test_file, 'rb') as f:
                transcript = client.audio.transcriptions.create(
                    model=whisper_api_model,
                    file=f,
                    language=default_language,
                    response_format="verbose_json",
                    timestamp_granularities=["segment"]
                )
            transcription_time = time.time() - start_time

            print(f"Temps de transcription : {transcription_time:.1f}s")
            print(f"Texte : {transcript.text}")

            if hasattr(transcript, 'segments') and transcript.segments:
                print(f"\nSegments avec timestamps :")
                for seg in transcript.segments:
                    # TranscriptionSegment est un objet avec des attributs, pas subscriptable
                    start = getattr(seg, 'start', seg.get('start') if hasattr(seg, 'get') else 0)
                    end = getattr(seg, 'end', seg.get('end') if hasattr(seg, 'get') else 0)
                    text = getattr(seg, 'text', seg.get('text') if hasattr(seg, 'get') else '')
                    print(f"  [{start:6.1f}s - {end:6.1f}s] {text.strip()}")

            transcription_results["api"] = {
                "text": transcript.text,
                "time": transcription_time
            }

        # Ecouter le fichier original
        print(f"\nEcoute du fichier original :")
        display(Audio(filename=str(test_file)))
else:
    print("Transcription desactivee (pas de fichiers de test)")

### Interpretation : Transcription avec horodatage

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Ratio temps reel | >10x typique (GPU) | Transcription bien plus rapide que la duree audio |
| Precision timestamps | ~0.1s segments | Suffisant pour le sous-titrage standard |
| Detection langue | >95% probabilite | Whisper detecte fiablement le francais |

**Points cles** :
1. Les word-level timestamps ne sont disponibles qu'en mode local
2. L'API est plus simple mais offre moins de controle
3. Le ratio temps reel depend fortement du GPU et de la taille du modele

## Section 2 : Transcription batch

En production, il est courant de transcrire de nombreux fichiers. Un pipeline batch doit gerer :

| Aspect | Description |
|--------|-------------|
| Decouverte | Lister les fichiers audio dans un repertoire |
| Filtrage | Verifier les formats supportes |
| Progression | Afficher l'avancement et les estimations de temps |
| Erreurs | Gerer les echecs sans bloquer le pipeline |
| Sortie | Sauvegarder les resultats de maniere structuree |

In [ ]:
# Pipeline de transcription batch
print("TRANSCRIPTION BATCH")
print("=" * 50)

SUPPORTED_FORMATS = {'.mp3', '.wav', '.flac', '.ogg', '.m4a', '.webm'}

def batch_transcribe(input_dir: Path, output_dir: Path,
                     language: str = "fr") -> List[Dict[str, Any]]:
    """Transcrit tous les fichiers audio d'un repertoire."""
    results = []

    # Decouverte des fichiers
    audio_files = [
        f for f in sorted(input_dir.iterdir())
        if f.suffix.lower() in SUPPORTED_FORMATS
    ]

    if not audio_files:
        print("Aucun fichier audio trouve")
        return results

    print(f"Fichiers trouves : {len(audio_files)}")
    total_start = time.time()

    for i, audio_file in enumerate(audio_files):
        print(f"\n[{i+1}/{len(audio_files)}] {audio_file.name}")
        file_start = time.time()

        try:
            if use_local_whisper and whisper_local_available:
                model = WhisperModel(whisper_model, device=actual_whisper_device, compute_type=actual_compute_type)
                segments, info = model.transcribe(str(audio_file), language=language)
                segments_list = list(segments)
                text = " ".join([s.text.strip() for s in segments_list])
                duration = info.duration
            else:
                with open(audio_file, 'rb') as f:
                    transcript = client.audio.transcriptions.create(
                        model=whisper_api_model, file=f, language=language
                    )
                text = transcript.text
                duration = 0  # API ne retourne pas la duree en mode simple

            file_time = time.time() - file_start

            result = {
                "file": audio_file.name,
                "text": text,
                "duration": duration,
                "transcription_time": file_time,
                "word_count": len(text.split()),
                "status": "success"
            }
            results.append(result)

            # Sauvegarder la transcription
            if save_output_files:
                txt_path = output_dir / f"{audio_file.stem}.txt"
                txt_path.write_text(text, encoding='utf-8')

            print(f"  Duree : {duration:.1f}s | Temps : {file_time:.1f}s | Mots : {len(text.split())}")
            print(f"  Texte : {text[:80]}...")

        except Exception as e:
            results.append({
                "file": audio_file.name,
                "status": "error",
                "error": str(e)[:100]
            })
            print(f"  ERREUR : {str(e)[:80]}")

    total_time = time.time() - total_start
    success = sum(1 for r in results if r['status'] == 'success')
    print(f"\nBatch termine : {success}/{len(audio_files)} reussis en {total_time:.1f}s")

    return results

# Executer le batch
if generate_audio and test_audio_files:
    batch_results = batch_transcribe(OUTPUT_DIR, OUTPUT_DIR, language=default_language)

    # Recapitulatif
    if batch_results:
        print(f"\nRecapitulatif batch :")
        print(f"{'Fichier':<30} {'Statut':<10} {'Mots':<8} {'Temps (s)':<10}")
        print("-" * 58)
        for r in batch_results:
            if r['status'] == 'success':
                print(f"{r['file']:<30} {r['status']:<10} {r['word_count']:<8} {r['transcription_time']:<10.1f}")
            else:
                print(f"{r['file']:<30} {r['status']:<10} {'N/A':<8} {'N/A':<10}")
else:
    batch_results = []
    print("Batch desactivee (pas de fichiers de test)")

### Interpretation : Transcription batch

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Debit batch | Variable selon GPU | L'initialisation du modele est le goulot d'etranglement |
| Gestion erreurs | Try/except par fichier | Un echec ne bloque pas le pipeline |
| Sauvegarde | .txt par fichier | Format simple et universel |

> **Note technique** : En production, il est preferable de charger le modele Whisper une seule fois et de transcrire tous les fichiers avec la meme instance.

## Section 3 : Diarisation de locuteurs

La diarisation identifie "qui parle quand" dans un enregistrement. C'est essentiel pour les reunions et interviews.

| Approche | Complexite | Precision | Prerequis |
|----------|-----------|-----------|----------|
| pyannote.audio | Elevee | Excellente | GPU, HuggingFace token |
| Energie simple | Faible | Basique | Aucun |
| Whisper segments | Moyenne | Correcte | Silences entre locuteurs |

Nous implementons ici une approche simplifiee basee sur l'analyse d'energie pour illustrer le concept.

In [ ]:
# Diarisation simplifiee basee sur l'energie
print("DIARISATION DE LOCUTEURS")
print("=" * 50)

import numpy as np

def simple_energy_diarization(audio_path: str, window_ms: int = 500,
                               silence_threshold: float = 0.02) -> List[Dict]:
    """Diarisation simplifiee basee sur l'energie du signal.

    Detecte les segments de parole et les silences pour
    estimer les tours de parole.
    """
    from pydub import AudioSegment

    audio = AudioSegment.from_file(audio_path)
    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)
    samples = samples / np.max(np.abs(samples))  # Normalisation

    sr = audio.frame_rate
    window_samples = int(sr * window_ms / 1000)

    # Calculer l'energie par fenetre
    segments = []
    is_speaking = False
    speaker_id = 0
    segment_start = 0

    for i in range(0, len(samples) - window_samples, window_samples):
        window = samples[i:i + window_samples]
        energy = np.sqrt(np.mean(window ** 2))  # RMS energy

        if energy > silence_threshold:
            if not is_speaking:
                segment_start = i / sr
                is_speaking = True
        else:
            if is_speaking:
                segment_end = i / sr
                # Nouveau segment de parole
                if segment_end - segment_start > 0.3:  # Min 300ms
                    segments.append({
                        "speaker": f"Speaker_{speaker_id % 2}",
                        "start": segment_start,
                        "end": segment_end,
                        "duration": segment_end - segment_start
                    })
                    speaker_id += 1
                is_speaking = False

    return segments

if generate_audio and test_audio_files:
    test_file = test_audio_files.get("reunion")

    if test_file and test_file.exists():
        print(f"Fichier : {test_file.name}")
        diarization_segments = simple_energy_diarization(str(test_file))

        print(f"\nSegments detectes : {len(diarization_segments)}")
        print(f"{'Locuteur':<15} {'Debut (s)':<12} {'Fin (s)':<12} {'Duree (s)':<10}")
        print("-" * 49)
        for seg in diarization_segments:
            print(f"{seg['speaker']:<15} {seg['start']:<12.1f} {seg['end']:<12.1f} {seg['duration']:<10.1f}")

        # Statistiques par locuteur
        speakers = set(s['speaker'] for s in diarization_segments)
        print(f"\nStatistiques par locuteur :")
        for speaker in sorted(speakers):
            speaker_segs = [s for s in diarization_segments if s['speaker'] == speaker]
            total_dur = sum(s['duration'] for s in speaker_segs)
            print(f"  {speaker} : {len(speaker_segs)} segments, {total_dur:.1f}s total")

        print(f"\n(Diarisation simplifiee - en production, utiliser pyannote.audio)")
    else:
        print("Fichier de test non disponible")
        diarization_segments = []
else:
    diarization_segments = []
    print("Diarisation desactivee")

## Section 4 : Generation de sous-titres SRT/VTT

Les formats de sous-titres standard permettent l'affichage synchronise avec la video :

| Format | Extension | Utilisation | Specifites |
|--------|-----------|-------------|------------|
| SRT (SubRip) | .srt | Universel | Simple, largement supporte |
| VTT (WebVTT) | .vtt | Web/HTML5 | Style CSS, regions, alignement |

### Format SRT
```
1
00:00:01,000 --> 00:00:04,500
Bienvenue dans cette presentation
```

### Format VTT
```
WEBVTT

00:00:01.000 --> 00:00:04.500
Bienvenue dans cette presentation
```

In [ ]:
# Generation de sous-titres SRT et VTT
print("GENERATION DE SOUS-TITRES")
print("=" * 50)

def format_timestamp_srt(seconds: float) -> str:
    """Formate un timestamp en format SRT (HH:MM:SS,mmm)."""
    td = timedelta(seconds=seconds)
    hours = int(td.total_seconds() // 3600)
    minutes = int((td.total_seconds() % 3600) // 60)
    secs = int(td.total_seconds() % 60)
    ms = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{ms:03d}"

def format_timestamp_vtt(seconds: float) -> str:
    """Formate un timestamp en format VTT (HH:MM:SS.mmm)."""
    td = timedelta(seconds=seconds)
    hours = int(td.total_seconds() // 3600)
    minutes = int((td.total_seconds() % 3600) // 60)
    secs = int(td.total_seconds() % 60)
    ms = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}.{ms:03d}"

def transcription_to_srt(segments: List[Dict], max_chars: int = 60) -> str:
    """Convertit des segments transcrits en format SRT."""
    srt_lines = []
    for i, seg in enumerate(segments, 1):
        start_ts = format_timestamp_srt(seg['start'])
        end_ts = format_timestamp_srt(seg['end'])
        text = seg.get('text', '').strip()
        # Couper les lignes longues
        if len(text) > max_chars:
            mid = len(text) // 2
            space_pos = text.rfind(' ', 0, mid)
            if space_pos > 0:
                text = text[:space_pos] + '\n' + text[space_pos+1:]
        srt_lines.append(f"{i}\n{start_ts} --> {end_ts}\n{text}\n")
    return '\n'.join(srt_lines)

def transcription_to_vtt(segments: List[Dict]) -> str:
    """Convertit des segments transcrits en format WebVTT."""
    vtt_lines = ["WEBVTT\n"]
    for seg in segments:
        start_ts = format_timestamp_vtt(seg['start'])
        end_ts = format_timestamp_vtt(seg['end'])
        text = seg.get('text', '').strip()
        vtt_lines.append(f"{start_ts} --> {end_ts}\n{text}\n")
    return '\n'.join(vtt_lines)

# Generer les sous-titres a partir de la transcription
if generate_audio and test_audio_files:
    test_file = test_audio_files.get("presentation")

    if test_file and test_file.exists():
        # Transcrire avec timestamps
        print(f"Transcription de {test_file.name} pour sous-titres...")

        if use_local_whisper and whisper_local_available:
            model = WhisperModel(whisper_model, device=actual_whisper_device, compute_type=actual_compute_type)
            segments, info = model.transcribe(str(test_file), language=default_language)
            sub_segments = [{"start": s.start, "end": s.end, "text": s.text.strip()} for s in segments]
        else:
            with open(test_file, 'rb') as f:
                transcript = client.audio.transcriptions.create(
                    model=whisper_api_model, file=f, language=default_language,
                    response_format="verbose_json", timestamp_granularities=["segment"]
                )
            sub_segments = [{"start": getattr(s, 'start', 0), "end": getattr(s, 'end', 0), "text": getattr(s, 'text', '').strip()}
                           for s in (transcript.segments or [])]

        if sub_segments:
            # Generer SRT
            srt_content = transcription_to_srt(sub_segments)
            print(f"\n--- Format SRT ---")
            print(srt_content[:500])

            # Generer VTT
            vtt_content = transcription_to_vtt(sub_segments)
            print(f"--- Format VTT ---")
            print(vtt_content[:500])

            # Sauvegarder
            if save_output_files:
                srt_path = OUTPUT_DIR / "presentation.srt"
                srt_path.write_text(srt_content, encoding='utf-8')
                vtt_path = OUTPUT_DIR / "presentation.vtt"
                vtt_path.write_text(vtt_content, encoding='utf-8')
                print(f"\nSauvegarde : {srt_path.name}, {vtt_path.name}")
        else:
            print("Aucun segment avec timestamps disponible")
else:
    print("Generation de sous-titres desactivee")

### Interpretation : Generation de sous-titres

| Aspect | SRT | VTT |
|--------|-----|-----|
| Compatibilite | VLC, MPC, YouTube | Navigateurs web, HTML5 |
| Style | Aucun | CSS (couleur, position) |
| Separateur ms | Virgule (,) | Point (.) |
| Index | Obligatoire (1, 2, 3...) | Optionnel |

> **Note technique** : Pour des sous-titres de qualite broadcast, limiter chaque ligne a 42 caracteres et chaque bloc a 2 lignes maximum.

## Section 5 : Resume automatique de reunions

Le pipeline complet pour resumer une reunion :

```
Audio reunion -> Whisper (transcription) -> GPT (resume) -> Document structure
```

Le LLM recoit la transcription brute et produit un resume structure avec points cles, decisions et actions.

In [ ]:
# Resume automatique de reunions
print("RESUME AUTOMATIQUE DE REUNION")
print("=" * 50)

def summarize_meeting(transcript_text: str, llm_client, model: str) -> str:
    """Resume une transcription de reunion via LLM."""
    prompt = f"""
Analyse cette transcription de reunion et produis un resume structure :

1. RESUME (2-3 phrases)
2. POINTS CLES (liste a puces)
3. DECISIONS PRISES (liste numerotee)
4. ACTIONS A SUIVRE (tableau : Action | Responsable | Echeance)

Transcription :
{transcript_text}
"""
    response = llm_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=800
    )
    return response.choices[0].message.content

if generate_audio and test_audio_files:
    test_file = test_audio_files.get("reunion")

    if test_file and test_file.exists():
        # Transcrire la reunion
        print(f"Etape 1 : Transcription de {test_file.name}")
        start_time = time.time()

        if use_local_whisper and whisper_local_available:
            model = WhisperModel(whisper_model, device=actual_whisper_device, compute_type=actual_compute_type)
            segments, info = model.transcribe(str(test_file), language=default_language)
            meeting_text = " ".join([s.text.strip() for s in segments])
        else:
            with open(test_file, 'rb') as f:
                transcript = client.audio.transcriptions.create(
                    model=whisper_api_model, file=f, language=default_language
                )
            meeting_text = transcript.text

        transcription_time = time.time() - start_time
        print(f"Transcription : {len(meeting_text)} chars en {transcription_time:.1f}s")
        print(f"Texte : {meeting_text[:150]}...")

        # Resumer via LLM
        print(f"\nEtape 2 : Resume via {llm_model}")
        start_time = time.time()
        summary = summarize_meeting(meeting_text, client, llm_model)
        summary_time = time.time() - start_time

        print(f"Resume genere en {summary_time:.1f}s")
        print(f"\n--- Resume de la reunion ---")
        print(summary)
        print(f"--- Fin du resume ---")

        # Sauvegarder
        if save_output_files:
            summary_path = OUTPUT_DIR / "reunion_resume.md"
            summary_path.write_text(
                f"# Resume de reunion\n\n"
                f"Date : {datetime.now().strftime('%Y-%m-%d')}\n\n"
                f"{summary}\n\n"
                f"---\n\n"
                f"## Transcription complete\n\n{meeting_text}",
                encoding='utf-8'
            )
            print(f"\nSauvegarde : {summary_path.name}")
else:
    print("Resume desactive")

### Interpretation : Resume de reunion

| Etape | Temps typique | Cout (API) |
|-------|--------------|------------|
| Transcription (1h audio) | 30-60s (GPU local) | $0.36 (API) |
| Resume LLM | 3-5s | ~$0.01 (GPT-4o-mini) |
| Total pipeline | < 2 min | < $0.40 |

**Points cles** :
1. Le pipeline complet est realiste pour un usage quotidien
2. Le cout reste tres faible meme pour de longues reunions
3. La qualite du resume depend directement de la qualite de la transcription

In [ ]:
# Mode interactif - Transcrire votre propre fichier audio
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - TRANSCRIPTION PERSONNALISEE")
    print("=" * 50)
    print("\nEntrez le chemin d'un fichier audio a transcrire :")
    print("(Laissez vide pour passer)")

    try:
        user_path = input("\nChemin du fichier : ")

        if user_path.strip():
            audio_path = Path(user_path.strip())
            if audio_path.exists():
                print(f"\nTranscription de {audio_path.name}...")

                if use_local_whisper and whisper_local_available:
                    model = WhisperModel(whisper_model, device=actual_whisper_device, compute_type=actual_compute_type)
                    segments, info = model.transcribe(str(audio_path), language=default_language)
                    text = " ".join([s.text.strip() for s in segments])
                else:
                    with open(audio_path, 'rb') as f:
                        transcript = client.audio.transcriptions.create(
                            model=whisper_api_model, file=f, language=default_language
                        )
                    text = transcript.text

                print(f"\nTranscription :\n{text}")
                print(f"\nMots : {len(text.split())}")
            else:
                print(f"Fichier non trouve : {audio_path}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

In [ ]:
# Statistiques de session
print("STATISTIQUES DE SESSION")
print("=" * 50)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Whisper : {'local (' + whisper_model + ')' if use_local_whisper else 'API'}")
print(f"LLM resume : {llm_model}")

if transcription_results:
    for method, data in transcription_results.items():
        print(f"Transcription ({method}) : {data['time']:.1f}s")

if batch_results:
    success = sum(1 for r in batch_results if r['status'] == 'success')
    print(f"Batch : {success}/{len(batch_results)} fichiers transcrits")

if save_output_files:
    saved_files = list(OUTPUT_DIR.glob('*'))
    print(f"Fichiers sauvegardes : {len(saved_files)} dans {OUTPUT_DIR}")

print(f"\nPROCHAINES ETAPES")
print(f"1. Explorer la composition musicale (04-3-Music-Composition-Workflow)")
print(f"2. Synchroniser audio et video (04-4-Audio-Video-Sync)")

print(f"\nNotebook termine - {datetime.now().strftime('%H:%M:%S')}")